# Indice espectrales NBR en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccionar imagenes Landsat RS

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32719", scale = 30)\
                  .clip(zona_ee)

In [7]:
# Seleccionar una imagen Sentinel2
img_S2 = ee.Image("COPERNICUS/S2_SR/20190808T145729_20190808T145732_T19LCF")\
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .clip(zona_ee)

## Consultar nombre bandas y proyección

In [8]:
# Nombre de las bandas
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())
print("Nombre bandas S2: ",img_S2.bandNames().getInfo())

Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas S2:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [9]:
# Consultar Proyeccion
print("Proyeccion L8: ",img_L8.projection().getInfo())
print("Proyeccion S2: ",img_S2.select(['BLUE']).projection().getInfo())

Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion S2:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 300000, 0, -10, 8600020]}


In [10]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [11]:
# Visualizar 
Map.addLayer(img_L8, viz_es, name = "Landsat8 RS")
Map.addLayer(img_S2, viz_es, name = "Sentinel2 RS")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Determinar NBR

In [12]:
# Funcion para NBR
def Calc_NBR(image):
    nbr = image.expression("float(nir - swir2)/(nir + swir2)", {
        'nir': image.select('NIR'),
        'swir2': image.select('SWIR2')
    })
    return nbr.rename("NBR")

In [13]:
# Calcular indices con funcion
nbr_L8 = Calc_NBR(img_L8)
nbr_S2 = Calc_NBR(img_S2)

In [14]:
# Simbologia de NBR
viz_nbr = {
    'palette': ["#66FF66","#FF99FF","#66FFFF","#FFFF99",
                "#FF9966","#FF6600","#FF0000"],
    'min': -0.4,
    'max': 0.8,
    'bands': "NBR"
}

In [15]:
# Visualizar 
Map.addLayer(nbr_L8, viz_nbr, name = "Landsat8 NBR")
Map.addLayer(nbr_S2, viz_nbr, name = "Sentinel2 NBR")
Map

Map(bottom=562519.4000244141, center=[-12.91187919521466, -70.03389358520509], controls=(WidgetControl(options…

## Exportar Imagen

In [16]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [17]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [18]:
# Exportar imagen directorio
geemap.ee_export_image(nbr_L8, filename="nbr_L8.tif", scale=30, region=geometria, file_per_band=False)
geemap.ee_export_image(nbr_S2, filename="nbr_S2.tif", scale=10, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\nbr_L8.tif
Generating URL ...
An error occurred while downloading.
Total request size (98259616 bytes) must be less than or equal to 33554432 bytes.


In [19]:
# Exportar en google drive
geemap.ee_export_image_to_drive(nbr_L8, description='nbr_L8', folder='GEE_Python', region=geometria, scale=30)
geemap.ee_export_image_to_drive(nbr_S2, description='nbr_S2', folder='GEE_Python', region=geometria, scale=10)

Exporting nbr_L8 ...
Exporting nbr_S2 ...
